<a href="https://colab.research.google.com/github/Hyperspectral01/Natural_Language_Computing/blob/main/3.CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is made for learning purposes and is inspired from work at https://www.kaggle.com/code/priyankdl/word2vec-skipgraom-cbow

**CBOW**

Importing the dependencies

In [ ]:
!pip install torch==2.0.1 torchtext==0.15.2
!pip install portalocker>=2.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
#For the model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim

#For preparing the input to the model
from functools import partial
import numpy as np
import pandas as pd
from torchtext import datasets
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

If cuda is available:

In [ ]:
if torch.cuda.is_available():
  device=torch.device(type='cuda',index=0)
else:
  device=torch.device(type='cpu',index=0)

Now taking the IMDB Dataset from torchtext.datasets and playing around with it

In [ ]:
train_data=datasets.IMDB(split='train')

In [ ]:
print(type(train_data))

<class 'torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe'>


So basically you cannot affors to download the entire dataset, so you get a Sharding Filter Iter Data Pipe from the torchtext.datasets and this pipeline then gives you the text you need

In [ ]:
test_data=datasets.IMDB(split='test')

In [ ]:
print(type(test_data))

<class 'torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe'>


So now we have two pipelines one for train data and another one for the test data

Now we use this pipeline to take out the text and store it

Since we need only the reviews part of the dataset and not the label or the sentiment part of the dataset, lets go ahead and store it

In [ ]:
train_reviews=[]
for label,review in train_data:
  train_reviews.append(review)
  if (len(train_reviews)>=1000):
    break

#train_data is an iterable, sort of a pipeline

In [ ]:
print(train_reviews[0])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [ ]:
print(type(train_reviews))

<class 'list'>


In [ ]:
print(len(train_reviews)) #so basically 25000 review strings for training

1000


Similarly for test data

In [ ]:
test_reviews=[]
for label,review in test_data:
  test_reviews.append(review)
  if (len(test_reviews)>=1000):
    break

In [ ]:
print(test_reviews[0])

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to alway

In [ ]:
print(len(test_reviews))

1000


Now getting the tokenizer which comes from torchtext.data.utils
The advantage here is that once you have your strings you do not need to worry about normalising the text like lowercasing,stemming and lemitization,removing the stopwords,removing the punctuations and so on,tokenizer automatically does all that
if (basic_english):_basic_english_normalisation() is called which does that and then the normal split function works

In [ ]:
tokenizer=get_tokenizer("basic_english",language="en")

Now we try and proceed in the following manner:

input is ready->vocab->dataloader->model->train->test

Building the vocab

In [ ]:
str1=["Hello my name is Shrey Hello","Hello there"]
for token in map(tokenizer,str1):
  print(token)

print(tokenizer("Hello there my name is shrey again!!!"))

['hello', 'my', 'name', 'is', 'shrey', 'hello']
['hello', 'there']
['hello', 'there', 'my', 'name', 'is', 'shrey', 'again', '!', '!', '!']


In [ ]:
minimum_allowed_frequency=20

list_of_tokens_along_with_duplicates=[]
for review in train_reviews:
  list_of_tokens_along_with_duplicates.append(tokenizer(review))

print("The iterable is a list of lists wherein the inner list would be the list of tokens corresponding to that string at that place")

vocab=build_vocab_from_iterator(
    list_of_tokens_along_with_duplicates, #map(tokenizer,train_reviews)
    min_freq=20,
    specials=["<unk>"],
    special_first=False
)

print(type(vocab))

vocab.set_default_index(vocab["<unk>"])
print(vocab.lookup_indices(["<unk>"]))
print(vocab["<unk>"])




The iterable is a list of lists wherein the inner list would be the list of tokens corresponding to that string at that place
<class 'torchtext.vocab.vocab.Vocab'>
[1198]
1198


Basically the vocab is of the nature of a dictionary (but it is actually of the type torchtext.vocab.Vocab), where the key is the token string and the value is its index in the vocab

In [ ]:
print("The size of the vocab is ",vocab.__len__())

The size of the vocab is  1199


Creating a text pipeline

In [ ]:
def text_pipeline(review):
  return vocab.lookup_indices(tokenizer(review))

Creating the colate function for the dataloader

In [ ]:
window_size=4
context_window_size=9
batch_size=16

In [ ]:
def colate_function(batch,text_pipeline):
  input=[]
  ground_truth=[]

  for review in batch:
    tokens=text_pipeline(review)

    if (len(tokens)<context_window_size):
      continue
    else:
      for idx in range(len(tokens)-context_window_size+1):
        context_window=tokens[idx:idx+context_window_size]
        target=context_window.pop(window_size)
        input.append(context_window)
        ground_truth.append(target)

  input=torch.tensor(input,dtype=torch.long)
  ground_truth=torch.tensor(ground_truth,dtype=torch.long)

  return input,ground_truth

Now using the colate function to create a custom dataloader,
dataloaders are used to convert the normal input data types to tensors that can be understood by model,
secondly,it is used to create custom dataloaders using colate function

In [ ]:
train_dataloader=DataLoader(
    train_reviews,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=partial(colate_function,text_pipeline=text_pipeline)
)

In [ ]:
test_dataloader=DataLoader(
    test_reviews,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=partial(colate_function,text_pipeline=text_pipeline)
)

Now for the model part

In [ ]:
class CBOW(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.emb1=nn.Embedding(
        num_embeddings=vocab_size,
        embedding_dim=300,
        max_norm=1
    )

    self.linear1=nn.Linear(
        in_features=300,
        out_features=vocab_size
    )

  def forward(self,x):
    x=self.emb1(x)
    x=x.mean(axis=1)
    x=self.linear1(x)

    return x





Finally train one epoch and test for accuracy after one epoch

In [ ]:
def train_one_epoch(model,dataloader,optimiser,loss_function):
  model.train()

  running_loss=[]
  for i,batch_data in enumerate(dataloader):
    inputs=batch_data[0].to(device)
    target=batch_data[1].to(device)
    outputs=model(inputs)
    loss=loss_function(outputs,target)

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    running_loss.append(loss.item())

  print("The Loss for this epoch is ",np.mean(running_loss))




In [ ]:
def try_testing_after_one_epoch(model,dataloader,loss_function):
  model.eval()

  running_losses=[]
  for i,batch_data in enumerate(dataloader):
    inputs=batch_data[0]
    targets=batch_data[1]
    outputs=model(inputs)
    loss=loss_function(outputs,targets)
    running_losses.append(loss.item())

  print("The Evaluation on testing data after this epoch is ",np.mean(running_losses))

Creating the optimiser and the loss function

In [ ]:
model=CBOW(vocab.__len__()).to(device)

loss_function=nn.CrossEntropyLoss()
optimiser=optim.Adam(model.parameters(),lr=0.001)

Now finally the training part for epochs=1

In [ ]:
for i in range(5):  #Epochs=1
  print("Epochs number:",i+1)
  train_one_epoch(model,train_dataloader,optimiser,loss_function)
  try_testing_after_one_epoch(model,test_dataloader,loss_function)


Epochs number: 1
The Loss for this epoch is  6.656619927239796
The Evaluation on testing data after this epoch is  5.9288262866792225
Epochs number: 2
The Loss for this epoch is  5.460186655559237
The Evaluation on testing data after this epoch is  5.066986477564251
Epochs number: 3
The Loss for this epoch is  5.02956541757735
The Evaluation on testing data after this epoch is  4.882398423694429
Epochs number: 4
The Loss for this epoch is  4.955040621379065
The Evaluation on testing data after this epoch is  4.84975246399168
Epochs number: 5
The Loss for this epoch is  4.943546537369016
The Evaluation on testing data after this epoch is  4.841776219625322


Now to trim the model and check the similarities of various word embeddings

In [ ]:
for name,child in model.named_children():
    print(name,child)

emb1 Embedding(1199, 300, max_norm=1)
linear1 Linear(in_features=300, out_features=1199, bias=True)


In [ ]:
trimmed_model=model.emb1
print(trimmed_model)

Embedding(1199, 300, max_norm=1)


In [ ]:
emb1=trimmed_model(torch.tensor([vocab.lookup_indices(["film"])]).to(device))
emb2=trimmed_model(torch.tensor([vocab.lookup_indices(["movie"])]).to(device))
print(emb1.shape, emb2.shape)
cos=torch.nn.CosineSimilarity(dim=2)
print(cos(emb1,emb2))

torch.Size([1, 1, 300]) torch.Size([1, 1, 300])
tensor([[0.9924]], grad_fn=<SumBackward1>)
